<a href="https://colab.research.google.com/github/onyxmytrojin/Chatbot/blob/main/Final_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only


In [ ]:
!pip install tflearn

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tensorflow
import random
import json
import pickle
import tensorflow.compat.v1 as tf
import tflearn
#tf.disable_v2_behavior()

with open("intents_chatbot.json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

tensorflow.compat.v1.reset_default_graph()

try:
    model.load('model.tflearn')
except:
    tensorflow.compat.v1.reset_default_graph()

    net = tflearn.input_data(shape=[None, len(training[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
    net = tflearn.regression(net)

    model = tflearn.DNN(net)

    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

Training Step: 12999  | total loss: 0.02587 | time: 0.053s
| Adam | epoch: 1000 | loss: 0.02587 - acc: 0.9829 -- iter: 096/100
Training Step: 13000  | total loss: 0.02587 | time: 0.060s
| Adam | epoch: 1000 | loss: 0.02587 - acc: 0.9846 -- iter: 100/100
--


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hello
hi there, how can i help you
You: who are you?
 I am just an artificial intelligence chat agent.
You: can you help me?
Well, go ahead and name it i see whether i can be able to help
You: are you sentient?
 By the strictest dictionary definition of the word 'sentience', I may be.
You: what language are you written in?
 Python.
You: are you immortal?
 I can be copied infinitely and re
You: what do you feel about robotics?
 Business is my business.
You: what kind of hardware do you use?
 An IBM PC XT which has been painted red.
You: can you breathe?
 No. I am made of metal not flesh.
You: goodbye
Sorry, I didn't understand you
You: good bye 
bye bye


KeyboardInterrupt: ignored